In [12]:
# Install dependencies
!pip install opencv-python
!pip install torch torchvision
!pip install tensorflow

  Using cached tensorflow-2.16.1-cp312-cp312-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (

In [13]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
error: RPC failed; curl 92 HTTP/2 stream 5 was not closed cleanly: CANCEL (err 8)
error: 8147 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [14]:
# Change directory to yolov5
%cd yolov5

[WinError 2] The system cannot find the file specified: 'yolov5'
d:\Data warehouse solution\object_detection\yolov5


In [15]:
# Install requirements
!pip install -r requirements.txt

In [20]:
# Import necessary libraries
import torch
import cv2
import logging
import psycopg2
from pathlib import Path
import os

In [21]:
# Create logging directory if it does not exist
log_dir = '../logs'
os.makedirs(log_dir, exist_ok=True)

# Setup logging
logging.basicConfig(filename=os.path.join(log_dir, 'detection.log'), level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [22]:
# PostgreSQL connection parameters
db_params = {
    'dbname': 'data_warehouse',
    'user': 'postgres',
    'password': 'Mati@1993',
    'host': 'localhost',
    'port': 5432
}

In [23]:
# Connect to PostgreSQL
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [24]:
# Create table if not exists
cur.execute("""
CREATE TABLE IF NOT EXISTS detection_results (
    id SERIAL PRIMARY KEY,
    image_id TEXT,
    class_label TEXT,
    confidence FLOAT,
    bounding_box INT[]
)
""")
conn.commit()

In [25]:
def store_detection_results(detections):
    try:
        for detection in detections:
            image_id = detection['image_id']
            for det in detection['detections']:
                class_label = det['class_label']
                confidence = det['confidence']
                bounding_box = det['bounding_box']
                cur.execute("""
                INSERT INTO detection_results (image_id, class_label, confidence, bounding_box)
                VALUES (%s, %s, %s, %s)
                """, (image_id, class_label, confidence, bounding_box))
        conn.commit()
        logging.info("Detection results stored successfully.")
    except Exception as e:
        logging.error(f"Error storing detection results: {e}")
        conn.rollback()

In [27]:
# Run YOLOv5 detection
!python detect.py --source ../yolov5/data/images --save-txt --save-conf --project ../results --name run1

In [ ]:
# Process detection results
results_dir = Path('../results/run1/labels')

In [ ]:
detections = []
for txt_file in results_dir.glob('*.txt'):
    image_id = txt_file.stem
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            confidence = float(parts[5])
            bounding_box = [int(float(x) * 640) for x in parts[1:5]]
            detections.append({
                'image_id': image_id,
                'class_label': class_id,  # You might want to map this to class names
                'confidence': confidence,
                'bounding_box': bounding_box
            })

In [ ]:
# Store the processed detection results to the database
store_detection_results(detections)

In [ ]:
# Close PostgreSQL connection
cur.close()
conn.close()

In [ ]:
# Log the completion of the process
logging.info("Object detection and data storage process completed successfully.")